In [ ]:
# ------------------------------------------------------------------------------
# This is generated from https://ml.azure.com/pipelineruns/id/9f77d021-13d4-4398-b3b4-a0240a22c32f/0720b2c0-6961-4359-80a9-7a5e76f97f5f?wsid=/subscriptions/f92fab0a-38b9-44ed-b800-2ed55fa1b9d9/resourcegroups/Groot-Shared/workspaces/gjd-test&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
# To run this code, please install SDK by this command:
# !pip install azure-ml-component[notebooks] --extra-index-url https://azuremlsdktestpypi.azureedge.net/modulesdkpreview  --user --upgrade
# More detailed guide to set up your environment: https://github.com/Azure/DesignerPrivatePreviewFeatures/blob/master/azure-ml-components/samples/setup-environment.ipynb
# ------------------------------------------------------------------------------

In [56]:
from azureml.core import Workspace
from azure.ml.component import Pipeline, Component, dsl

In [57]:
# configure aml workspace

subscription_id = 'f92fab0a-38b9-44ed-b800-2ed55fa1b9d9' 
workspace_name = 'gjd-test' 
resource_group = 'Groot-Shared'


ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

In [58]:
# get dataset
from azureml.core import Dataset
imagenet_train_dataset = Dataset.get_by_name(ws, name='ImageNet_Train', version=1)
imagenet_valid_dataset = Dataset.get_by_name(ws, name='ImageNet_Val', version=1)

In [59]:
# get components
convert_to_image_directory_func = Component.load(ws, name='azureml://Convert to Image Directory')
apply_image_transformation_func = Component.load(ws, name='azureml://Apply Image Transformation')
init_image_transformation_func = Component.load(ws, name='azureml://Init Image Transformation')

In [60]:
init_image_transformation_func = Component.load(ws, name='microsoft.sample.init-image-transformation')
module = init_image_transformation_func()
module.outputs.output_image_transformation.configure(datastore='datastore_name')

In [52]:
# define pipeline
@dsl.pipeline(name='test-cosmos-itp-image-classification', description='itp-cosmos-imageclassification', default_compute_target='eus-p40-5-gjd', default_datastore='searchdmparner')
def generated_pipeline():
    convert_to_image_directory_0 = convert_to_image_directory_func(
        input_dataset=imagenet_train_dataset)
    
    convert_to_image_directory_1 = convert_to_image_directory_func(
        input_dataset=imagenet_valid_dataset)
 
    init_image_transformation_0 = init_image_transformation_func(
        resize='False',
        size=256,
        center_crop='False',
        crop_size=224,
        pad='False',
        padding=0,
        color_jitter=False,
        grayscale=False,
        random_resized_crop='False',
        random_resized_crop_size=256,
        random_crop='False',
        random_crop_size=224,
        random_horizontal_flip=True,
        random_vertical_flip=False,
        random_rotation='False',
        random_rotation_degrees=0,
        random_affine='False',
        random_affine_degrees=0,
        random_grayscale=False,
        random_perspective=False)
    
    apply_image_transformation_0 = apply_image_transformation_func(
        input_image_transformation=init_image_transformation_0.outputs.output_image_transformation,
        input_image_directory=convert_to_image_directory_0.outputs.output_image_directory,
        mode='For training')

    apply_image_transformation_1 = apply_image_transformation_func(
        input_image_transformation=init_image_transformation_0.outputs.output_image_transformation,
        input_image_directory=convert_to_image_directory_1.outputs.output_image_directory,
        mode='For inference')

In [53]:
# create a pipeline
pipeline = generated_pipeline()

In [54]:
# validate pipeline and visualize the graph
pipeline.validate()

<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_decb71ae-33b4-4c7c-b449-5c32e1b21db9_widget', env_json='{"subscription…

{'result': 'validation passed', 'errors': []}

In [55]:
# submit a pipeline run
from azure.ml.component.dsl._version import VERSION

pipeline.submit(experiment_name='cosmos-datastore-test-image-classification', tags={'mode':'SDK/dsl','SDK-version':f'{VERSION}'})
#.wait_for_completion()

Submitted PipelineRun c2bd399e-a713-4d4b-8f3a-4a35c738f2c0
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/cosmos-datastore-test-image-classification/runs/c2bd399e-a713-4d4b-8f3a-4a35c738f2c0?wsid=/subscriptions/f92fab0a-38b9-44ed-b800-2ed55fa1b9d9/resourcegroups/Groot-Shared/workspaces/gjd-test


Experiment,Id,Type,Status,Details Page,Docs Page
cosmos-datastore-test-image-classification,c2bd399e-a713-4d4b-8f3a-4a35c738f2c0,azureml.PipelineRun,NotStarted,Link to Azure Machine Learning studio,Link to Documentation
